# Neural Networks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Neural Networks with scikit-learn
There is a basic implementation of neural networks in scikit-learn than you can use for prototyping and getting familiar with the concepts. Because it doesn't have GPU support, it's not really appropriate for larger applications.

Neural networks are implemented in the ``MLPClassifier`` and ``MLPRegressor`` classes in the ``sklearn.neural_networks`` module.
There are two standard solvers implemented: ``'adam'`` (the default) and ``'l-bfgs'``. ``'l-bfgs'`` might work better on very small toy datasets.

### Task 1
Create a synthetic "two moons" dataset, and visualize the decision boundary when varying the number of hidden units, the non-linearity, and the random state.
What do you notice?

In [ ]:
def plot_2d_classification(classifier, X, ax=None):                                       
                                                                                                            
    eps = X.std() / 2.
    if ax is None:
        ax = plt.gca()                                                                                                           
    
    # create a dense grid of values to evaluate the classifier
    x_min, x_max = X[:, 0].min() - eps, X[:, 0].max() + eps                                                                      
    y_min, y_max = X[:, 1].min() - eps, X[:, 1].max() + eps                                                                      
    xx = np.linspace(x_min, x_max, 1000)                                                                                         
    yy = np.linspace(y_min, y_max, 1000)                                                                                         
    X1, X2 = np.meshgrid(xx, yy)                                                                                                 
    X_grid = np.c_[X1.ravel(), X2.ravel()]
    # evaluate the classifier on the grid
    decision_values = classifier.predict(X_grid)
    # show the classification results:
    ax.imshow(decision_values.reshape(X1.shape), extent=(x_min, x_max,                                                           
                                                       y_min, y_max),                                                          
            aspect='auto', origin='lower', alpha=.5)                                                               
    ax.set_xlim(x_min, x_max)                                                                                                    
    ax.set_ylim(y_min, y_max)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=100, noise=0.25, random_state=3)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(10,), random_state=0).fit(X_train, y_train)
# todo: create subplots with multiple random states, different non-linearities and different numbers of hidden units
plot_2d_classification(mlp, X)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, s=60, edgecolor='k')

### Task 2
Apply ``MLPRegressor`` to the boston housing dataset. Does it do better or worse than Ridge (yesterday).

Try the model with and without scaling the data.

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

boston = load_boston()
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, random_state=0)

# ... solution here

In [ ]:
rng = np.random.RandomState(0)
x = np.sort(rng.uniform(size=100))
y = np.sin(10 * x) + 5 * x + np.random.normal(0, .3, size=100)
plt.plot(x, y, 'o')

In [ ]:
line = np.linspace(0, 1, 100)
X = x.reshape(-1, 1)

In [ ]:
from sklearn.neural_network import MLPRegressor
mlp_relu = MLPRegressor(solver="lbfgs").fit(X, y)
mlp_tanh = MLPRegressor(solver="lbfgs", activation='tanh').fit(X, y)

In [ ]:
plt.plot(x, y, 'o')
plt.plot(line, mlp_relu.predict(line.reshape(-1, 1)), label="relu")
plt.plot(line, mlp_tanh.predict(line.reshape(-1, 1)), label="tanh")
plt.legend()

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(
    digits.data / 16., digits.target, stratify=digits.target, random_state=0)

In [ ]:
mlp = MLPClassifier(max_iter=1000, random_state=0).fit(X_train, y_train)
print(mlp.score(X_train, y_train))
print(mlp.score(X_test, y_test))

## Keras
Now let's work with the much more flexible and efficient Keras library (which is using TensorFlow in the backend)
A simple way to work with keras is to build up models using the ``Sequential`` class.

The model below corresponds to mulinomial logistic regression. Run it on the ``digits`` dataset.


In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
# We divide the data by 16 to make sure it's on a reasonable scale (between zero and one)
X_train, X_test, y_train, y_test = train_test_split(digits.data / 16., digits.target, random_state=0)

import keras
num_classes = 10
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([
    Dense(10, input_shape=(64,)),
    Activation('softmax'),
])

In [ ]:
model.compile("adam", "categorical_crossentropy", metrics=['accuracy'])

In [ ]:
model.summary()

In keras, the ``fit`` method takes options like the number of epochs (iterations) to run, and whether to use a validation set for early stopping.

In [ ]:
model.fit(X_train, y_train, epochs=10, verbose=1, validation_split=.1)

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss: {:.3f}".format(score[0]))
print("Test Accuracy: {:.3f}".format(score[1]))

Here is a nice way to visualize the history of training the model:

In [ ]:
def plot_history(logger):
    df = pd.DataFrame(logger.history)
    df[['acc', 'val_acc']].plot()
    plt.ylabel("accuracy")
    df[['loss', 'val_loss']].plot(linestyle='--', ax=plt.twinx())
    plt.ylabel("loss")

In [ ]:
df = pd.DataFrame(history_callback.history)
df[['acc', 'val_acc']].plot()
plt.ylabel("accuracy")
df[['loss', 'val_loss']].plot(linestyle='--', ax=plt.twinx())
plt.ylabel("loss")

### Task
Add an additional hidden layer with a ``relu`` non-linearity and fit the new model (``Dense(20, activation='relu')``. Experiment with different layer sizes and different numbers of additional layers.